# Data Preparation Workflow

This notebook demonstrates the data preparation workflow for the Snow Drought Index package. It covers loading data, preprocessing, station extraction and filtering, and data availability assessment.

In [ ]:
# Install missing packages
#%pip install bottleneck
#%pip install git+https://github.com/Nadiesenali/snowdroughtindex-main   # Install snowdroughtindex from the local source directory
#%pip install --force-reinstall git+https://github.com/Nadiesenali/snowdroughtindex-main

# Import required packages
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from shapely.geometry import Point
import sys
from sklearn.metrics import mean_squared_error
import datetime
from pathlib import Path
from scipy.interpolate import interp1d
import random

# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))


# Import snowdroughtindex package
from snowdroughtindex.core import data_preparation

First, we'll load the SWE data and other required datasets.

In [ ]:
# Define data paths
canswe_path = project_root / 'data' / 'input_data' /'CanSWE'/'CanSWE-CanEEN_1928-2024_v7.nc'
CaSR_path = project_root / 'data' / 'output_data' /'casr_data'/'bow_combined_data.csv'
basin_path = project_root / 'data' / 'input_data' /'Elevation'/'Bow_elevation_combined.shp'
output_data = project_root / 'data' /'output_data'/'FROSTBYTE//'
output_plots = project_root / 'data' /'output_plots'/'FROSTBYTE//'

In [ ]:
# Load data using the implemented functions
canswe = data_preparation.load_swe_data(canswe_path)
casr_data = pd.read_csv(CaSR_path)
bow_basin = data_preparation.load_basin_data(basin_path)

# Extract CanSWE data for stations within the Bow basin

In [ ]:
# Get station locations from canswe
stations_df = pd.DataFrame({
	'station_id': canswe['station_id'].values,
	'lat': canswe['lat'].values,
	'lon': canswe['lon'].values
})

# Create Point geometries for each station
stations_gdf = gpd.GeoDataFrame(
	stations_df,
	geometry=gpd.points_from_xy(stations_df['lon'], stations_df['lat']),
	crs=bow_basin.crs
)

# Find stations within any of the Bow basin polygons
stations_in_basin = stations_gdf[stations_gdf.within(bow_basin.unary_union)]

# Select these stations from canswe
bow_canswe = canswe.sel(station_id=stations_in_basin['station_id'].values)

# Convert to DataFrame
#bow_canswe_df = bow_canswe.to_dataframe().reset_index()

# Save the extracted data to a nc file
#bow_canswe.to_netcdf(output_data + 'bow_canswe.nc')


# Extract CaSRv3.1 SWE data within the Bow basin

In [ ]:
# CaSR dataframe
casr_df = pd.DataFrame(casr_data)

# kee SWE, 'date', 'Grid_id', 'Elevation_category
casr_df = casr_df[['time', 'Grid_id','lat', 'lon', 'Elevation_Category','SWE']]

display(casr_df.head())


In [ ]:
# convert to geodataframe
casr_gdf = gpd.GeoDataFrame(
    casr_df,
    geometry=gpd.points_from_xy(casr_df['lon'], casr_df['lat']),
    crs=bow_basin.crs
)
display(casr_gdf.head())

In [ ]:
# Plot Bow basin polygons colored by elevation category
fig, ax = plt.subplots(figsize=(10, 10))

# Create elevation class column based on 'mean' elevation
bins = [500, 1000, 1500, 2000, 2500]
labels = ['500_1000m', '1000_1500m', '1500_2000m', '2000_2500m']
bow_basin['elev_class'] = pd.cut(bow_basin['mean'], bins=bins, labels=labels, include_lowest=True, right=False)

# Ensure elev_class is a categorical type with desired order
bow_basin['elev_class'] = pd.Categorical(
    bow_basin['elev_class'],
    categories=labels,
    ordered=True
)

bow_basin.plot(ax=ax, column='elev_class', cmap='PuBu', legend=True, edgecolor='black')
# Add station points
stations_in_basin.plot(ax=ax, color='red', markersize=20, label='CanSWE Stations')
casr_gdf.plot(ax=ax, color='orange', markersize=5, label='CaSR grid points')
ax.set_title('CanSWE data stations')
ax.grid(True, linestyle='--', alpha=0.5)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()
plt.savefig(str(output_plots / 'bow_basin_by_elev_class.png'), dpi=300)
plt.show()

## 1. FROSTBYTE gap filling

FROSTBYTE workflow  (In the original workflow, SCDNA precipitation data were also used to increase the data availability. However, here I'm only using CanSWE data because I couldn't find compatible precipitation dataset for the time period 1980-2024)

In [ ]:
# Set user-specified variables
flag_buffer_default, buffer_km_default = 0, 0 # buffer flag (0: no buffer around test basin, 1: buffer of value buffer_default around test basin) and buffer default value in km to be applied if flag = 1
month_start_water_year_default, day_start_water_year_default = 10, 1  # water year start
month_end_water_year_default, day_end_water_year_default = 9, 30  # water year end
min_obs_corr_default = 3 # the minimum number of overlapping observations required to calculate the correlation between 2 stations
min_obs_cdf_default = 10 # the minimum number of observations required to calculate a station's cdf
min_corr_default = 0.6 # the minimum correlation value required for donor stations to be selected
window_days_default = 7 # the number of days used on either side of the infilling date for gap filling calculations
min_obs_KGE_default = 3 # the minimum number of observations required to calculate the KGE''
max_gap_days_default = 15  # max. number of days for gaps allowed in the daily SWE data for the linear interpolation
artificial_gap_perc_default = 100 # the percentage of observations to remove during the artificial gap filling for each station & month's first day
iterations_default = 1 # the number of times we repeat the artificial gap filling
artificial_gap_filling_flag = 0 # indicates whether artificial gap filling is performed (1) or not (0)
artificial_gap_filling_basins = ['all'] # a list of the basin(s) to run the gap filling for. To include all basins simply write 'all'

In [ ]:
def artificial_gap_filling(original_data, iterations, artificial_gap_perc, window_days, min_obs_corr, min_obs_cdf, min_corr, min_obs_KGE, flag):

    """Creating random artificial gaps in the original dataset for each month & station, and running the gap filling function to assess its performance.

    Keyword arguments:
    ------------------
    - original_data: Pandas DataFrame of original stations' observations dataset, to which data will be removed for artificial gap filling
    - iterations: Positive integer denoting the number of times we want to repeat the artificial gap filling (we remove data at random each time in the original dataset)
    - artificial_gap_perc: Percentage between 1 and 100 for the amount of data to remove for each station & month's first day
    - window_days: Positive integer denoting the number of days to select data for around a certain doy, to calculate correlations
    - min_obs_corr: Positive integer for the minimum number of overlapping observations required to calculate the correlation between 2 stations
    - min_obs_cdf: Positive integer for the minimum number of stations required to calculate a station's cdf
    - min_corr: Value between 0 and 1 for the minimum correlation value required to keep a donor station
    - min_obs_KGE: Positive integer for the minimum number of stations required to calculate a station's cdf
    - flag: Integer to plot the gap filled values vs the observed values (1) or not (0)

    Returns:
    --------
    - evaluation: Dictionary containing the artificial gap filling evaluation results for several metrics for each month's first day, station & iteration
    - fig (optional): A figure of the gap filled vs. the actual SWE observations for each first day of the month

    """
    # suppresses the "SettingWithCopyWarning"
    pd.set_option("mode.chained_assignment", None)

    # Set up the figure
    if flag == 1:
        ncols = 3
        fig, axs = plt.subplots(4, ncols, sharex=False, sharey=False, figsize=(8,10))
        plot_col = -1
        row = 0

    # Identify stations for gap filling (without P & external SWE stations (buffer) as we don't do any gap filling for these)
    cols = [c for c in original_data.columns if 'precip' not in c and 'ext' not in c]

    # Create an empty dictionary to store the metric values for each month, station & iteration
    evaluation = {}
    metrics = ['RMSE', "KGE''", "KGE''_corr", "KGE''_bias", "KGE''_var"]
    for m in metrics:
        evaluation[m] = np.ones((12, len(cols), iterations)) * np.nan

    # Calculate correlations between stations that have overlapping observations
    corr = calculate_stations_doy_corr(original_data, window_days, min_obs_corr)

    # loop over months
    for mo in range(1,12+1):

        # controls for plotting on right subplot
        if flag == 1:
            plot_col += 1
            if plot_col == ncols:
                row += 1
                plot_col = 0

        # loop over iterations
        for i in range(iterations):

            # initialize counter to assign results to the right station
            elem = -1

            # looping over stations
            for s in cols:

                # update counter to assign results to the right station
                elem += 1

                # duplicate original data to create artificial gaps from this
                artificial_gaps_data = original_data.copy()

                # remove all missing values for a given station for which to perform gap filling
                station_nomissing_values = pd.DataFrame(artificial_gaps_data[s].dropna())

                # add DOY to select data to gap fill within a time window around first day of month
                station_nomissing_values['doy'] = station_nomissing_values.index.dayofyear

                # calculate the doy corresponding to the date - using 2010 as common year (not leap year)
                doy = int(datetime.datetime(2010,mo,1).strftime('%j'))

                # calculate the start & end doys of the time window for quantile mapping, with caution around the start & end of the calendar year
                window_startdoy = (doy-window_days)%365
                window_startdoy = 365 if window_startdoy == 0 else window_startdoy
                window_enddoy = (doy+window_days)%365
                window_enddoy = 366 if window_enddoy == 0 or window_enddoy == 365 else window_enddoy

                # select data within time window
                if window_startdoy > window_enddoy:
                    data_window = station_nomissing_values[(station_nomissing_values['doy']>=window_startdoy) | (station_nomissing_values['doy'] <= window_enddoy)]
                else:
                    data_window = station_nomissing_values[(station_nomissing_values['doy']>=window_startdoy) & (station_nomissing_values['doy'] <= window_enddoy)]

                # Select target data within this time window
                data_window_target = data_window[s]

                # calculate the number of observations to remove for this station & month's first day
                n = int(len(data_window.index) * artificial_gap_perc / 100)

                # if the number of observations is above zero we can proceed with the gap filling
                if n > 0:

                    # randomly select n dates from the station's data (no duplicates) and remove them from the original dataset - if 100% is removed then all dates will be selected
                    if artificial_gap_perc == 100:
                        dates_to_remove = data_window.index
                    else:
                        dates_to_remove = data_window.index[random.sample(range(0, len(data_window.index)), n)]
                    artificial_gaps_data[s].loc[dates_to_remove] = np.nan
                    artificial_gaps_data = artificial_gaps_data.loc[dates_to_remove]

                    # Keep only SWE station to gap fill
                    gapfilled_data = artificial_gaps_data[s].copy()

                    # Identify dates for gap filling
                    time_index = data_window.dropna().index

                    # Loop over dates for gap filling
                    for d in time_index:

                        # Get the doy corresponding to the date
                        doy = data_window.dropna().loc[d,'doy']

                        # Get IDs of all stations with data for this date (and within time window)
                        data_window_allstations = artificial_gaps_data.dropna(axis=1, how='all')
                        non_missing_stations = [c for c in data_window_allstations.columns]
                        data_window_allstations['days_to_date'] = abs((d - data_window_allstations.index).days)

                        # We can continue if there are enough target data to build cdf
                        if len(data_window_target.index) >= min_obs_cdf:

                            # Get ids of all stations with correlations >= a minimum correlation for this doy, not including the target station itself
                            non_missing_corr = corr[doy][s].dropna()
                            non_missing_corr = non_missing_corr[non_missing_corr.index.isin(non_missing_stations)]
                            potential_donor_stations = non_missing_corr[non_missing_corr >= min_corr].index.values
                            potential_donor_stations = [c for c in potential_donor_stations if s not in c]

                            # If there is at least one potential donor station, proceed
                            if len(potential_donor_stations) > 0:

                                # Sort the donor stations from highest to lowest value
                                potential_donor_stations_sorted = corr[doy].loc[potential_donor_stations,s].dropna().sort_values(ascending=False).index.values

                                # Loop over sorted donor stations until I find one with enough data to build a cdf
                                for donor_station in potential_donor_stations_sorted:

                                    # Select data within time window for this doy from all years
                                    data_window_donor = data_window_allstations[donor_station].dropna()

                                    # We can continue if there are enough donor data to build cdf
                                    if len(data_window_donor.index) >= min_obs_cdf:

                                        # If the donor station has multiple values within the window, we keep the closest donor station value to the date we are gap filling
                                        sorted_data_window = data_window_allstations.sort_values(by=['days_to_date'])
                                        value_donor = sorted_data_window[donor_station].dropna()[0]

                                        # Perform the gap filling using quantile mapping
                                        value_target = quantile_mapping(data_window_donor, data_window_target, value_donor, min_obs_cdf, flag=0)

                                        if value_target != None:
                                            gapfilled_data.loc[d] = value_target

                                        break

                                    else:
                                        continue

                    # combine observed & predicted data into a single Pandas dataframe
                    # results = gapfilled_data[s].loc[dates_to_remove]
                    results = gapfilled_data.to_frame(name='pre')
                    results['obs'] = original_data[s].loc[dates_to_remove]
                    results = results.dropna()

                    # plot the gap filled vs the observed values
                    if flag == 1:
                        axs[row,plot_col].scatter(results['obs'], results['pre'], color='b', alpha=.3)
                        axs[row,plot_col].set_title('month'+str(mo))
                        if row == 3 and plot_col == 0:
                            axs[row,plot_col].set_xlabel('observed')
                            axs[row,plot_col].set_ylabel('infilling')

                    # if there are no predicted values set the metrics to nan
                    if results.empty == True:
                        for m in metrics:
                            evaluation[m][mo-1,elem,i] = np.nan

                    # otherwise proceed with evaluating the gap filling performance
                    else:
                        rmse = mean_squared_error(results['obs'], results['pre'], squared=False)
                        kge_prime_prime = KGE_Tang2021(results['obs'].values, results['pre'].values, min_obs_KGE)
                        evaluation['RMSE'][mo-1,elem,i] = rmse
                        evaluation["KGE''"][mo-1,elem,i] = kge_prime_prime['KGE']
                        evaluation["KGE''_corr"][mo-1,elem,i] = kge_prime_prime['r']
                        evaluation["KGE''_bias"][mo-1,elem,i] = kge_prime_prime['beta']
                        evaluation["KGE''_var"][mo-1,elem,i] = kge_prime_prime['alpha']

                # else if the number of observations is zero we go to the next station
                else:
                    continue

    if flag == 1:
        plt.tight_layout()
        return evaluation, fig

    else:
        return evaluation

###

def KGE_Tang2021(obs, pre, min_obs_KGE):

    """Calculates the modified Kling-Gupta Efficiency (KGE") and its 3 components.
    The KGE measures the correlation, bias and variability of the simulated values against the observed values.
    KGE" was proposed by Tang et al. (2021) to solve issues arising with 0 values in the KGE or KGE'.
    For more info, see https://doi.org/10.1175/jcli-d-21-0067.1
    KGE" range: -Inf to 1. Perfect score: 1. Units: Unitless.
    Correlation (r): Perfect score is 1.
    Bias ratio (beta): Perfect score is 0.
    Variability ratio (alpha):  Perfect score is 1.

    Keyword arguments:
    ------------------
    - obs: Numpy Array of observations to evaluate
    - pre: Numpy Array of predictions/simulations to evaluate
    - min_obs_KGE: Positive integer for the minimum number of stations required to calculate a station's cdf

    Returns:
    --------
    - KGEgroup: Dictionary containing the final KGE'' value as well as all elements of the KGE''

    """

    ind_nan = np.isnan(obs) | np.isnan(pre)
    obs = obs[~ind_nan]
    pre = pre[~ind_nan]

    if len(obs) >= min_obs_KGE:

        pre_mean = np.mean(pre, axis=0, dtype=np.float64)
        obs_mean = np.mean(obs, axis=0, dtype=np.float64)
        pre_std = np.std(pre, axis=0)
        obs_std = np.std(obs, dtype=np.float64)

        # Check to see if all forecast values are the same. If they are r cannot be calculated and is set to 0
        # For more info: https://doi.org/10.5194/hess-23-4323-2019 (Section 2)
        if pre_std == 0:

            r = 0

        else:

            r = np.sum((pre - pre_mean) * (obs - obs_mean), axis=0, dtype=np.float64) / \
                np.sqrt(np.sum((pre - pre_mean) ** 2, axis=0, dtype=np.float64) *
                        np.sum((obs - obs_mean) ** 2, dtype=np.float64))

        alpha = pre_std / obs_std

        beta = (pre_mean - obs_mean) / obs_std

        KGE = 1 - np.sqrt((r - 1) ** 2 + (alpha - 1) ** 2 + (beta) ** 2)

        KGEgroup = {'KGE': KGE, 'r': r, 'alpha': alpha, 'beta': beta}

    else:

        KGEgroup = {'KGE': np.nan, 'r': np.nan, 'alpha': np.nan, 'beta': np.nan}

    return KGEgroup

###
def data_availability_monthly_plots_1(SWE_stations, original_SWE_data, gapfilled_SWE_data, flag):

    """Calculating and plotting the % of SWE stations available on the first day of each month of each year.

    Keyword arguments:
    ------------------
    - SWE_stations: Pandas GeoDataFrame of all SWE stations
    - original_SWE_data: xarray DataArray of the original SWE observations
    - gapfilled_SWE_data: xarray DataArray of the SWE observations after gap filling
    - flag: Flag to indicate if gap filled data was provided (1) or not (0). In the case that it is provided, a comparison plot will be made to compare data availability in the original data vs the gap filled data

    Returns:
    --------
    - Bar chart timeseries of SWE stations available on the first day of each month of each year

    """

    # Initialize plot
    fig, axs = plt.subplots(6, 2, sharex=True, sharey=True, figsize=(14,8))
    elem = -1
    column = 0

    # Loop over months
    for m in range(1,12+1):

        # controls for plotting on right subplot (i.e., month)
        elem += 1
        if elem == 6:
            column += 1
            elem = 0

        # for SWE data with gap filling
        if flag == 1:

            # extract data on the first of the month m
            data_month_gapfilled = gapfilled_SWE_data.sel(station_id=SWE_stations.station_id.values, time=( (gapfilled_SWE_data['time.month'] == m) & (gapfilled_SWE_data['time.day'] == 1) ))

            # count the % of stations with data on those dates
            data_month_gapfilled_count = data_month_gapfilled.count(dim='station_id') / len(SWE_stations) * 100

            # plot bar chart of available data
            axs[elem,column].bar(data_month_gapfilled_count['time.year'], data_month_gapfilled_count.data, color='r', alpha=.5)

        # same process as above but for original SWE data
        data_month = original_SWE_data.sel(station_id=SWE_stations.station_id.values, time=( (original_SWE_data['time.month'] == m) & (original_SWE_data['time.day'] == 1) ))
        data_month_count = data_month.count(dim='station_id') / len(SWE_stations) * 100
        axs[elem,column].bar(data_month_count['time.year'], data_month_count.data, color='b')

        # add plot labels
        if elem == 5 and column == 0:
            axs[elem,column].set_ylabel('% of SWE stations \n with data in basin')
        month_name = datetime.datetime.strptime(str(m), "%m").strftime("%b")
        axs[elem,column].set_title('1st '+month_name, fontweight='bold')

        if flag == 1:
            bluepatch = mpatches.Patch(color='b', label='original data')
            redpatch = mpatches.Patch(color='r', alpha=.5, label='after gap filling')
            plt.legend(handles=[bluepatch, redpatch])

    plt.tight_layout()

    return fig

###

def data_availability_monthly_plots_2(SWE_data):

    """Creating bar chart subplots of the days with SWE observations around the 1st day of each month.

    Keyword arguments:
    ------------------
    - SWE_data: Pandas DataFrame containing the SWE stations observations

    Returns:
    --------
    - Bar chart subplots of the days with SWE observations around the 1st day of each month

    """

    # Initialize plot
    fig, axs = plt.subplots(6, 2, sharex=False, sharey=True, figsize=(8,16))
    elem = -1
    column = 0

    # Add day of year (doy) to test basin SWE observations Pandas DataFrame
    SWE_data_with_doy = SWE_data.copy()
    SWE_data_with_doy['doy'] = SWE_data_with_doy.index.dayofyear

    # Remove automatic stations as they distract the analysis
    manual_stations = [s for s in SWE_data_with_doy.columns if s[-1] != 'P']
    SWE_data_with_doy_manual = SWE_data_with_doy[manual_stations]

    # Define the doys of 1st of each month
    doys_first_month = [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]

    # Loop over months
    for m in range(1,12+1):

        # controls for plotting on right subplot
        elem += 1
        if elem == 6:
            column += 1
            elem = 0

        # calculate the start & end of the data selection window, with caution around the start & end of the calendar year
        window_start = (doys_first_month[m-1]-15)%366
        if window_start == 0:
            window_start = 365
        window_end = (doys_first_month[m-1]+15)%366
        if window_end == 0 or window_end == 365:
            window_end = 366

        # select SWE observations within window
        if window_start > window_end:
            data_window = SWE_data_with_doy_manual[(SWE_data_with_doy_manual['doy']>=window_start) | (SWE_data_with_doy_manual['doy'] <= window_end)]
        else:
            data_window = SWE_data_with_doy_manual[(SWE_data_with_doy_manual['doy']>=window_start) & (SWE_data_with_doy_manual['doy'] <= window_end)]

        # drop dates or stations with no data at all
        data_window = data_window.dropna(axis=0, how='all')
        data_window = data_window.dropna(axis=1, how='all')

        # count total number of stations with data on each doy
        stations_cols = [c for c in data_window.columns if 'doy' not in c]
        data_stations_window = data_window[stations_cols]
        data_count_window = data_stations_window.count(axis=1)

        # create xticks to plot the data for each doy
        if window_start > window_end:
            xticks = list(np.arange(window_start,365+1))+list(np.arange(1,window_end+1))
        else:
            xticks = list(np.arange(window_start,window_end+1))
        xticks_plot = np.arange(len(xticks))

        # save the data for the right doy
        data_count_plot = [0]*len(xticks)
        for x in range(len(data_window.index)):
            doy = data_window.iloc[x]['doy']
            if doy == 366:
                doy = 365
            data_count_plot[xticks.index(doy)] += data_count_window.iloc[x]

        # plot data
        axs[elem,column].bar(xticks_plot, data_count_plot, color='b')
        axs[elem,column].set_xticks([xticks_plot[0],xticks_plot[15],xticks_plot[-1]])
        axs[elem,column].set_xticklabels([xticks[0],doys_first_month[m-1],xticks[-1]])

        # add plot labels
        if elem == 5 and column == 0:
            axs[elem,column].set_ylabel('# of SWE obs.')
            axs[elem,column].set_xlabel('DOY')

        if elem == 5 and column == 1:
            axs[elem,column].set_xlabel('DOY')

        month_name = datetime.datetime.strptime(str(m), "%m").strftime("%b")
        axs[elem,column].set_title('1st '+month_name+' +/- 15 days', fontweight='bold')

    plt.tight_layout()

    return fig

###
def calculate_stations_doy_corr(stations_obs, window_days, min_obs_corr):

    """Calculating stations' correlations for each day of the year (doy; with a X-day window centered around the doy).

    Keyword arguments:
    ------------------
    - stations_obs: Pandas DataFrame of all SWE & P stations observations
    - window_days: Positive integer denoting the number of days to select data for around a certain doy, to calculate correlations
    - min_obs_corr: Positive integer for the minimum number of overlapping observations required to calculate the correlation between 2 stations

    Returns:
    --------
    - stations_doy_corr: Dictionary containing a Pandas DataFrame of stations correlations for each day of year

    """

    # Set up the dictionary to save all correlations
    stations_doy_corr = {}

    # Duplicate the stations observations Pandas DataFrame and add doy column
    stations_obs_doy = stations_obs.copy()
    stations_obs_doy['doy'] = stations_obs_doy.index.dayofyear

    # Loop over days of the year
    for doy in range(1,366+1):

        # calculate the start & end of the data selection window, with caution around the start & end of the calendar year
        window_start = (doy-window_days)%366
        window_start = 366 if window_start == 0 else window_start
        window_end = (doy+window_days)%366
        window_end = 366 if window_end == 0 else window_end

        # select data for the window of interest
        if window_start > window_end:
            data_window = stations_obs_doy[(stations_obs_doy['doy']>=window_start) | (stations_obs_doy['doy'] <= window_end)]
        else:
            data_window = stations_obs_doy[(stations_obs_doy['doy']>=window_start) & (stations_obs_doy['doy'] <= window_end)]

        # calculate the Pearson product-moment correlations between stations if the minimum number of observations criterium is met
        data_window = data_window.drop(columns=['doy'])
        corr = data_window.corr(method='spearman', min_periods=min_obs_corr)
        # np.fill_diagonal(corr.values, np.nan)

        # save correlation for the doy to the dictionary
        stations_doy_corr[doy] = corr

    return stations_doy_corr

###

def qm_gap_filling(original_data, window_days, min_obs_corr, min_obs_cdf, min_corr):

    """Performing the gap filling for all missing observations (when possible) using quantile mapping.
    For each target station and each date for which date is missing, we identify a donor stations as the station with:
    - data for this date,
    - a cdf for this doy,
    - and the best correlation to the target station (correlation >= min_corr for this doy).

    Keyword arguments:
    ------------------
    - original_data: Pandas DataFrame of original stations' observations dataset, which will be gap filled
    - window_days: Positive integer denoting the number of days to select data for around a certain doy, to calculate correlations
    - min_obs_corr: Positive integer for the minimum number of overlapping observations required to calculate the correlation between 2 stations
    - min_obs_cdf: Positive integer for the minimum number of stations required to calculate a station's cdf
    - min_corr: Value between 0 and 1 for the minimum correlation value required to keep a donor station

    Returns:
    --------
    - gapfilled_data: Pandas DataFrame of gap filled stations' observations
    - data_type_flags: Pandas DataFrame with information about the type of data (estimates or observations) in the gap filled dataset
    - donor_stationIDs: Pandas DataFrame with information about the donor station used to fill each of the gaps

    """

    # Create a duplicate of the dataset to gap fill
    gapfilled_data = original_data.copy()

    # Remove P & external SWE stations (buffer) from the dataframe
    cols = [c for c in original_data.columns if 'precip' not in c and 'ext' not in c]

    # Keep only gap filled SWE stations (without P stations & external SWE stations)
    gapfilled_data = gapfilled_data[cols]

    # Add doy to the Pandas DataFrame
    original_data['doy'] = original_data.index.dayofyear

    # Set empty dataframes to keep track of data type and donor station ids
    data_type_flags = pd.DataFrame(data=0, index=original_data.index, columns=cols)
    donor_stationIDs = pd.DataFrame(data=np.nan, index=original_data.index, columns=cols)

    # Calculate correlations between stations that have overlapping observations
    corr = calculate_stations_doy_corr(original_data, window_days, min_obs_corr)

    # Identify dates for gap filling
    time_index = original_data.index

    # Loop over dates for gap filling
    for d in time_index:

        # Calculate the doy corresponding to the date
        # Note: doy 365 and 366 are bundled together
        doy = original_data.loc[d,'doy']

        # Calculate the start and end dates of the time window for the gap filling steps
        window_startdate = d - pd.Timedelta(days=window_days)
        window_enddate = d + pd.Timedelta(days=window_days)

        # Get IDs of all stations with data for this date (and within time window)
        data_window = original_data[window_startdate:window_enddate].dropna(axis=1, how='all')
        non_missing_stations = [c for c in data_window.columns if 'doy' not in c]
        data_window['days_to_date'] = abs((d - data_window.index).days)

        # Calculate the start & end doys of the time window for quantile mapping, with special rules around the start & end of the calendar year
        window_startdoy = (data_window['doy'].iloc[0])%366
        window_startdoy = 366 if window_startdoy == 0 else window_startdoy
        window_enddoy = (data_window['doy'].iloc[-1])%366
        window_enddoy = 366 if window_enddoy == 0 else window_enddoy

        # Loop over stations to gap fill
        for target_station in cols:

            # If station has no data, proceed with the gap filling
            if np.isnan(original_data.loc[d,target_station]):

                # Select target data within time window for this doy from all years
                if window_startdoy > window_enddoy:
                    data_window_target = original_data[target_station].dropna()[(original_data['doy']>=window_startdoy) | (original_data['doy'] <= window_enddoy)]
                else:
                    data_window_target = original_data[target_station].dropna()[(original_data['doy']>=window_startdoy) & (original_data['doy'] <= window_enddoy)]

                # We can continue if there are enough target data to build cdf
                if len(data_window_target.index) >= min_obs_cdf:

                    # Get ids of all stations with correlations >= a minimum correlation for this doy, not including the target station itself
                    non_missing_corr = corr[doy][target_station].dropna()
                    non_missing_corr = non_missing_corr[non_missing_corr.index.isin(non_missing_stations)]
                    potential_donor_stations = non_missing_corr[non_missing_corr >= min_corr].index.values
                    potential_donor_stations = [c for c in potential_donor_stations if target_station not in c]

                    # If there is at least one potential donor station, proceed
                    if len(potential_donor_stations) > 0:

                        # Sort the donor stations from highest to lowest value
                        potential_donor_stations_sorted = corr[doy].loc[potential_donor_stations,target_station].dropna().sort_values(ascending=False).index.values

                        # Loop over sorted donor stations until I find one with enough data to build a cdf
                        for donor_station in potential_donor_stations_sorted:

                            # Select data within time window for this doy from all years
                            if window_startdoy > window_enddoy:
                                data_window_donor = original_data[donor_station].dropna()[(original_data['doy'] >= window_startdoy) | (original_data['doy'] <= window_enddoy)]
                            else:
                                data_window_donor = original_data[donor_station].dropna()[(original_data['doy'] >= window_startdoy) & (original_data['doy'] <= window_enddoy)]

                            # We can continue if there are enough donor data to build cdf
                            if len(data_window_donor.index) >= min_obs_cdf:

                                # If the donor station has multiple values within the window, we keep the closest donor station value to the date we are gap filling
                                sorted_data_window = data_window.sort_values(by=['days_to_date'])
                                value_donor = sorted_data_window[donor_station].dropna()[0]

                                # Perform the gap filling using quantile mapping
                                value_target = quantile_mapping(data_window_donor, data_window_target, value_donor, min_obs_cdf, flag=0)

                                if value_target != None:
                                    gapfilled_data.loc[d,target_station] = value_target
                                    data_type_flags.loc[d,target_station] = 1
                                    donor_stationIDs.loc[d,target_station] = donor_station

                                break

                            else:
                                continue

    return gapfilled_data, data_type_flags, donor_stationIDs

###

def quantile_mapping(data_donor, data_target, value_donor, min_obs_cdf, flag):

    """Calculating target station's gap filling value from donor station's value using quantile mapping.

    Keyword arguments:
    ------------------
    - data_donor: Pandas DataFrame of donor station observations used to build empirical cdf
    - data_target: Pandas DataFrame of target station observations used to build empirical cdf
    - value_donor: Integer of donor station value used in the quantile mapping
    - min_obs_cdf: Positive integer for the minimum number of unique observations required to calculate a station's cdf
    - flag: Integer to plot (1) or not (0) the donor and target stations' cdfs

    Returns:
    --------
    - value_target: Integer of target station value calculated using quantile mapping
    - plot of the donor and target stations' cdfs (optional)

    """

    # build the donor station's empirical cdf
    sorted_data_donor = data_donor.drop_duplicates().sort_values(ignore_index=True)

    # build the target station's empiral cdf
    sorted_data_target = data_target.drop_duplicates().sort_values(ignore_index=True)

    # Calculate the donor & target stations' cdfs if they both have at least X unique observations
    if (len(sorted_data_donor) >= min_obs_cdf) & (len(sorted_data_target) >= min_obs_cdf):

        # Calculate the cumulative probability corresponding to the donor value
        rank_donor_obs = sorted_data_donor[sorted_data_donor == value_donor].index[0]
        total_obs_donor = len(sorted_data_donor)
        cumul_prob_donor_obs = (rank_donor_obs + 1) / total_obs_donor

        # Calculate the cumulative probability corresponding to the target value
        cumul_prob_target = np.arange(1,len(sorted_data_target)+1) / (len(sorted_data_target))

        # inter-/extrapolate linearly to get the target value corresponding to the donor station's cumulative probability
        inverted_edf = interp1d(cumul_prob_target, sorted_data_target, fill_value="extrapolate")
        value_target = round(float(inverted_edf(cumul_prob_donor_obs)),2)

        # set any potential negative values from interpolation/extrapolation to zero
        if(value_target) < 0:
            value_target = 0

        # if requested, plot the target & donor stations' cdfs
        if flag == 1:
            plt.figure()
            plt.plot(sorted_data_donor, np.arange(1,len(sorted_data_donor)+1) / (len(sorted_data_donor)), label='donor')
            plt.plot(sorted_data_target, cumul_prob_target, label='target')
            plt.scatter(value_donor, cumul_prob_donor_obs)
            plt.legend()

        return value_target

    # If either/both the target & donor stations have < X observations do nothing
    else:
        return None

###

def quantile_mapping(data_donor, data_target, value_donor, min_obs_cdf, flag):

    """Calculating target station's gap filling value from donor station's value using quantile mapping.

    Keyword arguments:
    ------------------
    - data_donor: Pandas DataFrame of donor station observations used to build empirical cdf
    - data_target: Pandas DataFrame of target station observations used to build empirical cdf
    - value_donor: Integer of donor station value used in the quantile mapping
    - min_obs_cdf: Positive integer for the minimum number of unique observations required to calculate a station's cdf
    - flag: Integer to plot (1) or not (0) the donor and target stations' cdfs

    Returns:
    --------
    - value_target: Integer of target station value calculated using quantile mapping
    - plot of the donor and target stations' cdfs (optional)

    """

    # build the donor station's empirical cdf
    sorted_data_donor = data_donor.drop_duplicates().sort_values(ignore_index=True)

    # build the target station's empiral cdf
    sorted_data_target = data_target.drop_duplicates().sort_values(ignore_index=True)

    # Calculate the donor & target stations' cdfs if they both have at least X unique observations
    if (len(sorted_data_donor) >= min_obs_cdf) & (len(sorted_data_target) >= min_obs_cdf):

        # Calculate the cumulative probability corresponding to the donor value
        rank_donor_obs = sorted_data_donor.reset_index(drop=True)[sorted_data_donor.values == value_donor].index[0]
        total_obs_donor = len(sorted_data_donor)
        cumul_prob_donor_obs = (rank_donor_obs + 1) / total_obs_donor

        # Calculate the cumulative probability corresponding to the target value
        cumul_prob_target = np.arange(1,len(sorted_data_target)+1) / (len(sorted_data_target))

        # inter-/extrapolate linearly to get the target value corresponding to the donor station's cumulative probability
        inverted_edf = interp1d(cumul_prob_target, sorted_data_target, fill_value="extrapolate")
        value_target = round(float(inverted_edf(cumul_prob_donor_obs)),2)

        # set any potential negative values from interpolation/extrapolation to zero
        if(value_target) < 0:
            value_target = 0

        # if requested, plot the target & donor stations' cdfs
        if flag == 1:
            plt.figure()
            plt.plot(sorted_data_donor, np.arange(1,len(sorted_data_donor)+1) / (len(sorted_data_donor)), label='donor')
            plt.plot(sorted_data_target, cumul_prob_target, label='target')
            plt.scatter(value_donor, cumul_prob_donor_obs)
            plt.legend()

        return value_target

    # If either/both the target & donor stations have < X observations do nothing
    else:
        return None

###

In [ ]:
# open bow_canswe.csv file
bow_canswe = pd.read_csv(output_data / 'bow_canswe.csv')

In [ ]:
# Re-organize the dataset as needed
# Restrict bow_canswe to 1980-01-01 through 2024-07-31
bow_canswe['time'] = pd.to_datetime(bow_canswe['time'])
SWE_stations_df = bow_canswe[(bow_canswe['time'] >= '1980-01-01') & (bow_canswe['time'] <= '2023-07-31')]

# If you want to convert this DataFrame to an xarray Dataset similar to the original code:
SWE_stations_ds = SWE_stations_df.set_index(['station_id', 'time']).to_xarray()

display(SWE_stations_ds)

In [ ]:
# dataframe of the SWE stations
bow_canswe_df = SWE_stations_ds.to_dataframe().reset_index()

# Extract unique station coordinates
unique_stations = bow_canswe_df.reset_index().drop_duplicates(subset='station_id')[['station_id', 'lon', 'lat']]

# Convert SWE stations DataArray to GeoDataFrame for further analysis
data = {'station_id': unique_stations['station_id'].values, 
        'lon': unique_stations['lon'].values, 
        'lat': unique_stations['lat'].values} 
df = pd.DataFrame(data)
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
crs = "EPSG:4326"
SWE_stations_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)

display(SWE_stations_gdf)

In [ ]:
# Convert test basin SWE data DataSet to Pandas DataFrame for further analysis
SWE_testbasin_df = SWE_stations_ds.to_dataframe().drop(columns=['lon','lat','station_name']).unstack()['snw'].T

# Remove time from dates
SWE_testbasin_df['date'] = SWE_testbasin_df.index.normalize()
SWE_testbasin_df = SWE_testbasin_df.set_index('date')

# Drop the dates with no data at all across all stations
SWE_testbasin_df = SWE_testbasin_df.dropna(axis=0, how='all')

# Choose data for the period of interest 1980 - 2024
SWE_testbasin_df = SWE_testbasin_df.loc['1980-01-01':'2023-07-31']

display(SWE_testbasin_df)

In [ ]:
# Plot timeseries of SWE station observations in the test basin
fig = plt.figure(figsize=(15,10))

for s in SWE_stations_ds.station_id.values:
    SWE_stations_ds.snw.sel(station_id = s).plot(marker='o', alpha=.3, markersize=1, label=s)

plt.title('')
plt.ylabel('SWE [mm]')
plt.xlabel('')
plt.xlim(pd.to_datetime('1980-01-01'), pd.to_datetime('2024-07-31'))
plt.ylim(0, 1200)
# Place legend at the bottom with 5 columns per line
plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.25), ncol=10, fontsize=8, frameon=False)
plt.tight_layout();

In [ ]:
# Plot data availabity monthly plots

data_availability = data_availability_monthly_plots_1(SWE_stations_ds, SWE_stations_ds.snw, None, flag=0)
# Save the plot
data_availability.savefig(output_plots / 'data_availability_monthly_plots_1.png', dpi=300)

In [ ]:
# Plot bar charts of the days with SWE observations around the first day of each month
fig = data_availability_monthly_plots_2(SWE_testbasin_df)

Connect CaSR SWE data to CanSWE data before linear interpolation 

In [ ]:
# Load CaSRv3.1 data
casr_data = pd.read_csv(CaSR_path)
# Convert 'time' column to datetime
casr_data['time'] = pd.to_datetime(casr_data['time'])
# Create a DataFrame from the CaSR data
casr_df = pd.DataFrame(casr_data)

display(casr_df.head())

In [ ]:
# rename Grid_id to station_id and keep SWE, 'time', 'station_id'
casr_df = casr_df[['time', 'Grid_id','SWE']]
casr_df.rename(columns={'Grid_id': 'station_id','time':'date'}, inplace=True)
display(casr_df.head())

In [ ]:
# Check column names to ensure 'date' exists
print(casr_df.columns)

# Pivot the CaSR DataFrame to have dates as index and station_id as columns
casr_df = casr_df.pivot(index='date', columns='station_id', values='SWE')

# Optionally, rename the columns to a simple format if needed
casr_df.columns = [f'SWE_{i+1}' for i in range(len(casr_df.columns))]

display(casr_df)

In [ ]:
# Visual check
casr_df['SWE_5']['1982-09-01':'1987-10-02'].plot(marker='o', color='b')
plt.tight_layout();

In [ ]:
SWE_stations_ds['time'] = pd.date_range(start='1980-01-01', periods=SWE_stations_ds.dims['time'], freq='D')
# Linear interpolation to fill in small data gaps
SWE_testbasin_interp_da = SWE_stations_ds.snw.interpolate_na(method='linear', dim='time', max_gap=datetime.timedelta(days=max_gap_days_default))
# Only drop columns that exist
df_interp = SWE_testbasin_interp_da.to_dataframe()
cols_to_drop = [col for col in ['lon', 'lat', 'station_name'] if col in df_interp.columns]
SWE_obs_basin_interp_df = df_interp.drop(columns=cols_to_drop).unstack()['snw'].T
SWE_obs_basin_interp_df['date'] = SWE_obs_basin_interp_df.index.normalize()
SWE_obs_basin_interp_df = SWE_obs_basin_interp_df.set_index('date')
display(SWE_obs_basin_interp_df)

In [ ]:
plt.figure(figsize=(8,3))
SWE_obs_basin_interp_df.iloc[:,10].plot(color='r', marker='o', ms=2, label='gap filled', lw=0, alpha=.3)
SWE_testbasin_df.iloc[:,10].plot(color='k', marker='o', ms=2, label='gap filled', lw=0, alpha=.3)
plt.xlabel('')
plt.ylabel('SWE [mm]')
plt.legend();

In [ ]:
# Save flags for linear interpolation to a new Pandas dataframe (observations = 0; estimates = 1)
flags_interp_testbasin_da = SWE_testbasin_interp_da.copy().fillna(-999)
original_da = SWE_stations_ds.snw.copy().fillna(-999)
flags_interp_testbasin_da = xr.where(flags_interp_testbasin_da==original_da, 0, 1)
# Only drop columns that exist in the DataFrame
cols_to_drop = [col for col in ['lon', 'lat', 'station_name'] if col in flags_interp_testbasin_da.to_dataframe().columns]
flags_interp_testbasin_df = flags_interp_testbasin_da.to_dataframe().drop(columns=cols_to_drop).unstack()['snw'].T
flags_interp_testbasin_df['date'] =  flags_interp_testbasin_df.index.normalize()
flags_interp_testbasin_df = flags_interp_testbasin_df.set_index('date')
display(SWE_testbasin_interp_da)

In [ ]:
# Merge test basin SWE and casr_df
SWE_testbasin_interp_df = SWE_obs_basin_interp_df.merge(casr_df, left_index=True, right_index=True, how='outer', suffixes=('_testbasin', '_casr'))

display(SWE_testbasin_interp_df)


In [ ]:
start_date = pd.to_datetime("1980-01-01")
end_date = pd.to_datetime("2023-07-31")
chunk_years = 5

# Initialize output dataframes
all_SWE_obs = []
all_flags = []
all_donors = []

current_start = start_date

while current_start < end_date:
    current_end = min(current_start + pd.DateOffset(years=chunk_years), end_date)
    
    print(f"Processing chunk: {current_start.date()} to {current_end.date()}")

    chunk_df = SWE_testbasin_interp_df.loc[current_start:current_end].copy()
    
    SWE_obs_chunk, flags_chunk, donors_chunk = qm_gap_filling(
        chunk_df,
        window_days=window_days_default,
        min_obs_corr=min_obs_corr_default,
        min_obs_cdf=min_obs_cdf_default,
        min_corr=min_corr_default
    )

    all_SWE_obs.append(SWE_obs_chunk)
    all_flags.append(flags_chunk)
    all_donors.append(donors_chunk)

    current_start = current_end + pd.DateOffset(days=1)

# Combine the parts
SWE_obs_basin_gapfilled_df = pd.concat(all_SWE_obs)
flags_gapfill_basin_df = pd.concat(all_flags)
donor_stations_gapfill_basin_df = pd.concat(all_donors)

In [ ]:
# Combine linear interpolation and quantile mapping flags into a single Pandas dataframe
#if not flags_interp_basin_df.index.equals(flags_gapfill_basin_df.index) or not flags_interp_basin_df.columns.equals(flags_gapfill_basin_df.columns):
flags_basin_df = SWE_obs_basin_interp_df + flags_gapfill_basin_df

display(flags_basin_df)

In [ ]:
# Combine gap filled dataset and metadata into a single dataset
SWE_P_gapfill_testbasin_da = xr.DataArray(data=SWE_obs_basin_gapfilled_df.values, coords=dict(time=SWE_obs_basin_gapfilled_df.index.values, station_id=SWE_obs_basin_gapfilled_df.columns.values), dims=['time','station_id'], name='SWE', attrs={'long_name':'Surface snow water equivalent','units':'kg m**-2'})
flags_testbasin_da = xr.DataArray(data=flags_basin_df.values, coords=dict(time=flags_basin_df.index.values, station_id=flags_basin_df.columns.values), dims=['time','station_id'], name='flag', attrs={'description':'observations = 0; estimates = 1'})
donor_stations_gapfill_testbasin_da = xr.DataArray(data=donor_stations_gapfill_basin_df.values, coords=dict(time=donor_stations_gapfill_basin_df.index.values, station_id=donor_stations_gapfill_basin_df.columns.values), dims=['time','station_id'], name='donor_stations', attrs={'description':'station_id of donor stations used for gap filling'})
SWE_testbasin_gapfilled_ds = xr.merge([SWE_P_gapfill_testbasin_da, flags_testbasin_da, donor_stations_gapfill_testbasin_da])
# Only assign coordinates for station_ids present in SWE_stations_ds
station_ids_all = SWE_obs_basin_gapfilled_df.columns.values
station_ids_in_ds = [sid for sid in station_ids_all if sid in SWE_stations_ds.station_id.values]
station_ids_not_in_ds = [sid for sid in station_ids_all if sid not in SWE_stations_ds.station_id.values]

# Prepare arrays for lat, lon, and station_name, filling with NaN or empty string for missing stations
lats = []
lons = []
names = []
for sid in station_ids_all:
	if sid in SWE_stations_ds.station_id.values:
		lat_val = SWE_stations_ds.sel(station_id=sid).lat.values
		lon_val = SWE_stations_ds.sel(station_id=sid).lon.values
		name_val = SWE_stations_ds.sel(station_id=sid).station_name.values
		# If returned value is an array, get the first element
		if isinstance(lat_val, np.ndarray):
			lat_val = lat_val.item() if lat_val.size == 1 else lat_val[0]
		if isinstance(lon_val, np.ndarray):
			lon_val = lon_val.item() if lon_val.size == 1 else lon_val[0]
		if isinstance(name_val, np.ndarray):
			name_val = name_val.item() if name_val.size == 1 else name_val[0]
		lats.append(float(lat_val))
		lons.append(float(lon_val))
		names.append(str(name_val))
	else:
		lats.append(np.nan)
		lons.append(np.nan)
		names.append("")

SWE_testbasin_gapfilled_ds = SWE_testbasin_gapfilled_ds.assign_coords({'lat':('station_id',lats),'lon':('station_id',lons),'station_name':('station_id',names)})

display(SWE_testbasin_gapfilled_ds)
display(flags_testbasin_da)

In [ ]:
# Plot the first SWE station in the dataset to visually check gap filling results
plt.figure(figsize=(8,3))
SWE_obs_basin_gapfilled_df.iloc[:,10].plot(color='r', marker='o', ms=2, label='gap filled', lw=0, alpha=.3)
SWE_testbasin_interp_df.iloc[:,10].plot(color='k', marker='o', ms=2, label='original', lw=0)
plt.title(SWE_testbasin_interp_df.iloc[:,10].name)
plt.xlabel('')
plt.ylabel('SWE [mm]')
plt.legend();

plt.figure(figsize=(8,3))
SWE_obs_basin_gapfilled_df.iloc[:,15].plot(color='r', marker='o', ms=2, label='gap filled', lw=0, alpha=.3)
SWE_testbasin_interp_df.iloc[:,15].plot(color='k', marker='o', ms=2, label='original', lw=0)
plt.title(SWE_testbasin_interp_df.iloc[:,15].name)
plt.xlabel('')
plt.ylabel('SWE [mm]')
plt.legend();

In [ ]:
# Plot a bar chart of the number of times each donor station was used for infilling
count = []

for s in SWE_testbasin_interp_df.columns.values:
    count_s = SWE_testbasin_gapfilled_ds.donor_stations.where(SWE_testbasin_gapfilled_ds.donor_stations==s).count().data
    count.append(count_s)

fig = plt.figure()
plt.bar(SWE_testbasin_interp_df.columns.values, count, color='b')
plt.xticks(rotation=90)
plt.xlabel('Donor stations')
plt.ylabel('# times used for infilling')
plt.tight_layout();

In [ ]:
# Plot timeseries of the % of SWE stations with data in the test basin on the first day of each month, for the original data & after gap filling (flag=1)
fig = data_availability_monthly_plots_1(SWE_stations_ds, SWE_stations_ds.snw, SWE_testbasin_gapfilled_ds.SWE, flag=1)

In [ ]:
# Set the chunk size based on your preference or system capabilities
chunk_size = 5  # Adjust as needed based on your system's capacity

# Function to process data in chunks
def process_in_chunks(df, chunk_size):
    chunks = (df[i:i+chunk_size] for i in range(0, len(df), chunk_size))
    results = []
    for chunk in chunks:
        pd.set_option("mode.chained_assignment", None)  # Suppresses the "SettingWithCopyWarning"
        evaluation_artificial_gapfill_testbasin_dict, fig = artificial_gap_filling(chunk.copy(), iterations=iterations_default, artificial_gap_perc=artificial_gap_perc_default, window_days=window_days_default, min_obs_corr=min_obs_corr_default, min_obs_cdf=min_obs_cdf_default, min_corr=min_corr_default, min_obs_KGE=min_obs_KGE_default, flag=1)
        results.append((evaluation_artificial_gapfill_testbasin_dict, fig))
    return results

# Process your large dataframe in chunks
results = process_in_chunks(SWE_testbasin_interp_df, chunk_size)



In [ ]:
# Combine the evaluations
SWE_testbasin_evaluation_df = pd.concat(all_evaluations)
# Combine the figures
fig_combined = plt.figure(figsize=(15, 10))
for i, fig in enumerate(all_figs):
    ax = fig.get_axes()[0]  # Get the first axis of the figure
    ax.set_position([0.1 + (i % 3) * 0.3, 0.6 - (i // 3) * 0.3, 0.25, 0.25])  # Adjust position
    fig_combined.add_subplot(ax)
    ax.set_title(f"Chunk {i + 1}: {all_figs[i].axes[0].get_title()}", fontsize=10)
plt.tight_layout()


## 6. Summary

In this notebook, we've demonstrated the data preparation workflow for the Snow Drought Index package. We've loaded data, preprocessed it, extracted stations within the basin of interest, assessed data availability, and saved the processed data for use in subsequent analyses.

The workflow uses the following key functions from the `data_preparation` module:
- `load_swe_data()`, `load_precip_data()`, `load_basin_data()` for data loading
- `preprocess_swe()`, `preprocess_precip()` for data preprocessing
- `convert_to_geodataframe()` for converting data to GeoDataFrame
- `extract_stations_in_basin()` for extracting stations within a basin
- `filter_stations()` for filtering data by station
- `assess_data_availability()` for assessing data availability

These functions provide a standardized and reusable way to prepare data for the Snow Drought Index calculations.